In [1]:
import requests, sys, time, os

In [ ]:
key_path = 'api_key.txt'
output_dir = 'output/'
country_codes = ['US','GB','IN','DE','CA','FR','KR','RU','JP','BR','MX']
header = ['video_id', 'title', 'desc', 'categoryId', 'comments', 'tags', 'channel_title',  'view_count', 'comment_count']
MAX_VIDEOS = 100
MAX_COMMENTS = 100
GET_VIDEOS_COUNT = 100
GET_COMMENTS_COUNT = 100

In [72]:
class Video:
    def __init__(self, id, title, desc, category_id, comments, tags, channel_title, view_count, comment_count) -> None:
        self.id = id
        self.title = title
        self.desc = desc
        self.category_id = int(category_id)
        self.comments = comments
        self.tags = tags
        self.channel_title = channel_title
        self.view_count = view_count
        self.comment_count = comment_count
    
    def __str__(self):
        return f'{self.id},{self.title},{self.desc[:10]},{self.category_id},{len(self.comments)},{len(self.tags)},{self.channel_title},{self.view_count},{self.comment_count}'


In [74]:
def get_videos_dict(videos, api_key):
    videos_dict = {}
    for video in videos:
        video_id = video['id']
        snippet = video['snippet']
        title = snippet.get('title','')
        desc = snippet.get('description', '')
        category_id = snippet.get('categoryId','')
        tags = snippet.get('tags', ['[none]'])
        statistics = video['statistics']
        view_count = statistics.get('viewCount', 0)
        comment_count = 0
        if 'commentCount' in statistics:
            comment_count = statistics['commentCount']
        channel_title = snippet.get('channelTitle','')
        comments = []
        cur_comment_count = 0
        next_comment_token = '&'
        while next_comment_token is not None and cur_comment_count < MAX_COMMENTS:
            request = f'https://www.googleapis.com/youtube/v3/commentThreads?key={api_key}&textFormat=plainText&part=snippet&videoId={video_id}&maxResults={GET_VIDEOS_COUNT}&pageToken={next_comment_token}'
            comments_request = requests.get(request)
            if not comments_request or not hasattr(comments_request, 'status_code') or comments_request.status_code == 429:
                print("Temp-Banned due to excess requests, please wait and continue later")
                break
            comments_request = comments_request.json()
            comments_items = comments_request.get('items', [])
            comments.extend([c.get('snippet', {}).get('topLevelComment',{}).get('snippet', {}).get('textDisplay', '') for c in comments_items])
            next_comment_token = comments_request.get("nextPageToken", None)
            next_comment_token = f"&pageToken={next_comment_token}&" if next_comment_token is not None else next_comment_token
            cur_comment_count += GET_COMMENTS_COUNT
        video = Video(video_id, title, desc, category_id, comments, tags, channel_title, view_count, comment_count)
        videos_dict[video_id] = video
    return videos_dict

<a href='https://colab.research.google.com/github/cdwangco/SentimentAnalysisProject/blob/main/MLProjectYTSentimentAnalysis.ipynb' target='_parent'><img src='https://colab.research.google.com/assets/colab-badge.svg' alt='Open In Colab'/></a>

In [75]:
def api_request(page_token, api_key, region):
    # Builds the URL and requests the JSON from it
    country_code = region
    chart = 'mostPopular'
    next_page_token = '&'
    cur_video_count = 0
    while next_page_token is not None and cur_video_count < MAX_VIDEOS:
        request_url = f'https://www.googleapis.com/youtube/v3/videos?part=id,statistics,snippet{page_token}chart={chart}&regionCode={country_code}&maxResults={GET_VIDEOS_COUNT}&key={api_key}'
        request = requests.get(request_url)
        if request.status_code == 429:
            print('Temp-Banned due to excess requests, please wait and continue later')
            sys.exit()
        request = request.json()
        items = request.get('items', [])
        next_page_token = request.get("nextPageToken", None)
        next_page_token = f"&pageToken={next_page_token}&" if next_page_token is not None else next_page_token
        videos_dict = get_videos_dict(items, api_key)
        cur_video_count += GET_VIDEOS_COUNT
    return videos_dict


def get_video_data(region='US'):
    api_key = ""
    with open(key_path, 'r') as file:
        api_key = file.readline()
    
    videos_dict = {}
    if region == 'all':
        for c in country_codes:
            res = api_request('&', api_key, c)
            for v in res.keys():
                videos_dict[v] = res[v]
    else: 
        api_request('&', api_key, region)
    return videos_dict


In [76]:
# videos = get_video_data(region='all')

Temp-Banned due to excess requests, please wait and continue later
Temp-Banned due to excess requests, please wait and continue later
Temp-Banned due to excess requests, please wait and continue later
Temp-Banned due to excess requests, please wait and continue later
Temp-Banned due to excess requests, please wait and continue later
Temp-Banned due to excess requests, please wait and continue later
Temp-Banned due to excess requests, please wait and continue later
Temp-Banned due to excess requests, please wait and continue later
Temp-Banned due to excess requests, please wait and continue later
Temp-Banned due to excess requests, please wait and continue later
Temp-Banned due to excess requests, please wait and continue later
Temp-Banned due to excess requests, please wait and continue later
Temp-Banned due to excess requests, please wait and continue later
Temp-Banned due to excess requests, please wait and continue later
Temp-Banned due to excess requests, please wait and continue l

In [5]:
def get_response(sentence):
    url = 'http://10.0.0.162:8000/sentiment?text=' + sentence
    url = url.replace(' ', '%20')
    request = requests.get(url)
    return request.json()

In [6]:
# sentence_good = 'hi i think you\'re awesome'
# get_response(sentence_good)

{'score': 1}

In [7]:
# sentence_bad = 'hi i think you\'re lame'
# get_response(sentence_bad)

{'score': 0}